In [12]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [13]:
def load_data(page_numbers: list[int]):
    pages = [pd.read_json(f"data/pages_with_sentiment/page{page}.json") 
             for page in page_numbers]

    return pd.concat(pages)

In [ ]:
sia = SentimentIntensityAnalyzer()

def predict_sentiment(text: str):
    compound_score = sia.polarity_scores(text)["compound"]

    if compound_score >= 0:
        return "Positive"
    else:
        return "Negative"

In [15]:
def calculate_prediction_accuracy(data: pd.DataFrame):
    num_correct = len(data[data["sentiment"] == data["predicted_sentiment"]])
    return num_correct / len(data)

In [16]:
data = load_data([1, 2, 3, 4, 5, 6])
print("Loaded", len(data), "statements")

data["predicted_sentiment"] = data["text"].apply(predict_sentiment)

accuracy = calculate_prediction_accuracy(data)

print("The sentiment analyzer was able to predict with accuracy of {:.2f}".format(accuracy))

Loaded 88 statements
The sentiment analyzer was able to predict with accuracy of 0.59
